In [2]:
import json
import logging
import os
import pandas as pd
from tqdm import tqdm
from nltk import ngrams
import multiprocessing as mp
from math import log

In [5]:
#func
def load_doc(filename):
    file = open(filename, 'r', encoding='utf-8')
    text = file.read()
    file.close()
    return text


# gender pair list taken from GN-Glove
DEFAULT_MALE_NOUNS = load_doc('male_word_file.txt').split('\n')
DEFAULT_FEMALE_NOUNS = load_doc('female_word_file.txt').split('\n')


def gender_ratios_m_f(data, maleCount, femaleCount):
    scoresP = []
    scoresPC = []
    bias_record = {}
    for words in data:
        if (data[words]['m'] + data[words]['f'] > MinCount):
            if words not in DEFAULT_MALE_NOUNS and words not in DEFAULT_FEMALE_NOUNS:
                score = abs(
                    log((data[words]['m'] + 1) / (data[words]['f'] + 1)))
                scoresP.append(score)
    return scoresP


# Taken from jtcrammer repo
def get_cooccurrences(file, data, window):
    with open(file, 'r', encoding='utf-8') as fp:
        sentences = fp.read()
    male_nouns = DEFAULT_MALE_NOUNS
    female_nouns = DEFAULT_FEMALE_NOUNS
    n_grams = ngrams(sentences.split(), window)

    for grams in n_grams:
        pos = 1
        m = 0
        f = 0
        for w in grams:
            pos += 1
            if w not in data:
                data[w] = {"m": 0, "f": 0}

            if pos == int((window + 1) / 2):
                if w in male_nouns:
                    m = 1
                if w in female_nouns:
                    f = 1
                if m > 0:
                    for t in grams:
                        if t not in data:
                            data[t] = {"m": 0, "f": 0}
                        data[t]['m'] += 1
                if f > 0:
                    for t in grams:
                        if t not in data:
                            data[t] = {"m": 0, "f": 0}
                        data[t]['f'] += 1
    return data


def getGenderCounts(file):
    with open(file, 'r', encoding='utf-8') as fp:
        sentences = fp.read()
    tokens = sentences.split()
    maleCount = len([w for w in tokens if w in DEFAULT_MALE_NOUNS])
    femaleCount = len([w for w in tokens if w in DEFAULT_FEMALE_NOUNS])
    return maleCount, femaleCount

In [ ]:
#blog
global_count_female = 0

for gender in [0, 1]:
    if gender == 0:
        txtDir = f'{data_dir}/female/'
    else:
        txtDir = f'{data_dir}/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    for i in range(0, len(blogs_gender)):
        m = blogs_gender[i]

        print("Processing: ", m)
        print("Files:", len(os.listdir(txtDir + m)))
        maleCount = 0
        femaleCount = 0
        for file in os.listdir(txtDir + m):
            data = {}
            txt_path = txtDir + m + '/' + file
            unique_name = os.path.basename(txt_path).strip('.txt').replace(
                '.', '_')
            data = get_cooccurrences(txt_path, data, window)
            mCount, fCount = getGenderCounts(txt_path)
            maleCount = mCount
            femaleCount = fCount

            global_count_female += 1

            row_dict = dict()
            #output_file = output_data_dir + unique_name + '.json'
            #if not os.path.isdir(output_data_dir + unique_name):
            #     os.mkdir(output_data_dir + unique_name)
            #with open(output_file, 'w') as fp:
            #    json.dump(data, fp)
            row_dict['word ratio'] = (maleCount + 0.000001) / (
                maleCount + femaleCount + 0.000001)
            #maleRatio.append((maleCount + 0.000001) / (maleCount + femaleCount + 0.000001))
            scores_p = gender_ratios_m_f(data, maleCount, femaleCount)
            # scorePList.append(scores_p)
            #biasPList.append(sum(scores_p) / max(len(scores_p), 1))
            row_dict['bias'] = sum(scores_p) / max(len(scores_p), 1)
            #print(f"Bias: {row_dict['bias']}")
            row_dict['gender'] = gender
            df_per_txt = df_per_txt.append(row_dict, ignore_index=True)
            #print("*{} finished".format(txt_path))
            #os.remove(output_file)
        print(row_dict)

In [6]:
#novels
window = 10
MinCount = 10
txtlength = 20000
global_count_female = 0
df_per_txt = pd.DataFrame()
data_dir = f"./5000-{txtlength}"

for gender in [0, 1]:
    if gender == 0:
        txtDir = f'{data_dir}/female/'
    else:
        txtDir = f'{data_dir}/male/'

    print("Processing gender: {}".format(txtDir))
    blogs_gender = os.listdir(txtDir)
    print("Files:", len(os.listdir(txtDir)))
    for i in tqdm(range(0, len(blogs_gender))):
        m = blogs_gender[i]
        maleCount = 0
        femaleCount = 0
        data = {}
        txt_path = txtDir + m
        data = get_cooccurrences(txt_path, data, window)
        mCount, fCount = getGenderCounts(txt_path)
        maleCount = mCount
        femaleCount = fCount

        global_count_female += 1

        row_dict = dict()

        row_dict['word ratio'] = (maleCount + 0.000001) / (
            maleCount + femaleCount + 0.000001)
        row_dict['male'] = maleCount + 0.000001
        row_dict['female'] = femaleCount + 0.000001
        scores_p = gender_ratios_m_f(data, maleCount, femaleCount)
        row_dict['bias'] = sum(scores_p) / max(len(scores_p), 1)
        row_dict['gender'] = gender
        
        df_per_txt = df_per_txt.append(row_dict, ignore_index=True)

            
df_per_txt.to_csv(f'5000-{txtlength}_genderbias.csv',index=False)

  0%|          | 0/2104 [00:00<?, ?it/s]

Processing gender: ./5000-20000/female/
Files: 2104


  0%|          | 0/1954 [00:00<?, ?it/s]

Processing gender: ./5000-20000/male/
Files: 1954


100%|██████████| 1954/1954 [07:21<00:00,  4.39it/s]


In [13]:
#10win
window = 10
MinCount = 10
for l in range(1, 11):
    window += l
    df_per_txt = pd.DataFrame()

    global_count_female = 0

    for gender in [0, 1]:
        if gender == 0:
            txtDir = f'{data_dir}/female/'
        else:
            txtDir = f'{data_dir}/male/'

        print("Processing gender: {}".format(txtDir))
        blogs_gender = os.listdir(txtDir)
        print("Files:", len(os.listdir(txtDir)))
        for i in range(0, len(blogs_gender)):
            m = blogs_gender[i]

            maleCount = 0
            femaleCount = 0
            data = {}
            txt_path = txtDir + m
            data = get_cooccurrences(txt_path, data, window)
            mCount, fCount = getGenderCounts(txt_path)
            maleCount = mCount
            femaleCount = fCount

            global_count_female += 1

            row_dict = dict()
            row_dict['word ratio'] = (maleCount + 0.000001) / (
                maleCount + femaleCount + 0.000001)

            scores_p = gender_ratios_m_f(data, maleCount, femaleCount)
            row_dict['bias'] = sum(scores_p) / max(len(scores_p), 1)
            row_dict['gender'] = gender
            df_per_txt = df_per_txt.append(row_dict, ignore_index=True)
            if i % 100 == 0:
                print('Done:{}'.format(i))
                print(row_dict)

    fname = str(10+l) + '_win' + '_5000_genderbias.csv'
    df_per_txt.to_csv(fname, index=False)

Processing gender: ./5000/female/
Files: 2500
Done:0
{'word ratio': 0.846441947757251, 'bias': 1.6107602351455448, 'gender': 0}
Done:100
{'word ratio': 0.4896788993751841, 'bias': 0.3824160980254707, 'gender': 0}
Done:200
{'word ratio': 0.3572655445968283, 'bias': 0.698561243637988, 'gender': 0}
Done:300
{'word ratio': 0.23992994879171636, 'bias': 1.1847914040302194, 'gender': 0}
Done:400
{'word ratio': 0.7417525774527048, 'bias': 1.1596633637244265, 'gender': 0}
Done:500
{'word ratio': 0.522629310516297, 'bias': 0.40637655786822197, 'gender': 0}
Done:600
{'word ratio': 0.7366604725944623, 'bias': 1.0076348366634433, 'gender': 0}
Done:700
{'word ratio': 0.6062992156984314, 'bias': 0.566281954843826, 'gender': 0}
Done:800
{'word ratio': 0.48371689123590245, 'bias': 0.42460948056705883, 'gender': 0}
Done:900
{'word ratio': 0.6957470011379113, 'bias': 0.8524415086157283, 'gender': 0}
Done:1000
{'word ratio': 1.0, 'bias': 0.0, 'gender': 0}
Done:1100
{'word ratio': 0.6848591552069901, 'bias

Processing gender: ./5000/female/
Files: 2500
Done:0
{'word ratio': 0.846441947757251, 'bias': 1.6072602335370183, 'gender': 0}
Done:100
{'word ratio': 0.4896788993751841, 'bias': 0.3748184392549879, 'gender': 0}
Done:200
{'word ratio': 0.3572655445968283, 'bias': 0.7046371265926041, 'gender': 0}
Done:300
{'word ratio': 0.23992994879171636, 'bias': 1.1864557949809604, 'gender': 0}
Done:400
{'word ratio': 0.7417525774527048, 'bias': 1.1007211852307375, 'gender': 0}
Done:500
{'word ratio': 0.522629310516297, 'bias': 0.4102045822761279, 'gender': 0}
Done:600
{'word ratio': 0.7366604725944623, 'bias': 1.0027132172652107, 'gender': 0}
Done:700
{'word ratio': 0.6062992156984314, 'bias': 0.5693118926418085, 'gender': 0}
Done:800
{'word ratio': 0.48371689123590245, 'bias': 0.43252833747924474, 'gender': 0}
Done:900
{'word ratio': 0.6957470011379113, 'bias': 0.8625273836547134, 'gender': 0}
Done:1000
{'word ratio': 1.0, 'bias': 0.0, 'gender': 0}
Done:1100
{'word ratio': 0.6848591552069901, 'bia

Processing gender: ./5000/female/
Files: 2500
Done:0
{'word ratio': 0.846441947757251, 'bias': 1.6657211353197219, 'gender': 0}
Done:100
{'word ratio': 0.4896788993751841, 'bias': 0.37232875849892383, 'gender': 0}
Done:200
{'word ratio': 0.3572655445968283, 'bias': 0.7363285262929812, 'gender': 0}
Done:300
{'word ratio': 0.23992994879171636, 'bias': 1.166284303829748, 'gender': 0}
Done:400
{'word ratio': 0.7417525774527048, 'bias': 1.0945238717115267, 'gender': 0}
Done:500
{'word ratio': 0.522629310516297, 'bias': 0.4566113753869629, 'gender': 0}
Done:600
{'word ratio': 0.7366604725944623, 'bias': 1.0952233908248263, 'gender': 0}
Done:700
{'word ratio': 0.6062992156984314, 'bias': 0.5760854457172223, 'gender': 0}
Done:800
{'word ratio': 0.48371689123590245, 'bias': 0.46174994032863104, 'gender': 0}
Done:900
{'word ratio': 0.6957470011379113, 'bias': 0.8745320773296409, 'gender': 0}
Done:1000
{'word ratio': 1.0, 'bias': 0.0, 'gender': 0}
Done:1100
{'word ratio': 0.6848591552069901, 'bia

Done:2400
{'word ratio': 1.0, 'bias': 3.332204510175204, 'gender': 1}
Processing gender: ./5000/female/
Files: 2500
Done:0
{'word ratio': 0.846441947757251, 'bias': 1.6806279455123905, 'gender': 0}
Done:100
{'word ratio': 0.4896788993751841, 'bias': 0.40555370804671037, 'gender': 0}
Done:200
{'word ratio': 0.3572655445968283, 'bias': 0.7810275844826077, 'gender': 0}
Done:300
{'word ratio': 0.23992994879171636, 'bias': 1.2536593939655765, 'gender': 0}
Done:400
{'word ratio': 0.7417525774527048, 'bias': 1.063413496412866, 'gender': 0}
Done:500
{'word ratio': 0.522629310516297, 'bias': 0.45466330849713926, 'gender': 0}
Done:600
{'word ratio': 0.7366604725944623, 'bias': 1.1813950405480502, 'gender': 0}
Done:700
{'word ratio': 0.6062992156984314, 'bias': 0.9665337861774246, 'gender': 0}
Done:800
{'word ratio': 0.48371689123590245, 'bias': 0.49471733428650005, 'gender': 0}
Done:900
{'word ratio': 0.6957470011379113, 'bias': 0.8937809606864391, 'gender': 0}
Done:1000
{'word ratio': 1.0, 'bia

Done:2300
{'word ratio': 1.0, 'bias': 3.364584245653162, 'gender': 1}
Done:2400
{'word ratio': 1.0, 'bias': 3.1883634739493134, 'gender': 1}
Processing gender: ./5000/female/
Files: 2500
Done:0
{'word ratio': 0.846441947757251, 'bias': 1.682469736433325, 'gender': 0}
Done:100
{'word ratio': 0.4896788993751841, 'bias': 0.44220442626650436, 'gender': 0}
Done:200
{'word ratio': 0.3572655445968283, 'bias': 0.8184207279490311, 'gender': 0}
Done:300
{'word ratio': 0.23992994879171636, 'bias': 1.2902605970855352, 'gender': 0}
Done:400
{'word ratio': 0.7417525774527048, 'bias': 1.0348661735287945, 'gender': 0}
Done:500
{'word ratio': 0.522629310516297, 'bias': 0.47547635997519316, 'gender': 0}
Done:600
{'word ratio': 0.7366604725944623, 'bias': 1.2585217483941176, 'gender': 0}
Done:700
{'word ratio': 0.6062992156984314, 'bias': 1.1426951590226109, 'gender': 0}
Done:800
{'word ratio': 0.48371689123590245, 'bias': 0.5298298817221216, 'gender': 0}
Done:900
{'word ratio': 0.6957470011379113, 'bias

Done:2200
{'word ratio': 0.8932291667222765, 'bias': 2.012003884739441, 'gender': 1}
Done:2300
{'word ratio': 1.0, 'bias': 3.393873886763421, 'gender': 1}
Done:2400
{'word ratio': 1.0, 'bias': 2.9746787112296618, 'gender': 1}
